In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

import plotly.express as px
from create_data import *
pd.options.plotting.backend = "plotly"

In [2]:
dfeco = read_data_file()

loaded files


In [3]:
dfeco.groupby('device_id').agg({'ind_shutdown_mal_3D_next':['count', 'mean', 'sum']})

ind_shutdown_mal_3D_next                 
                             count      mean    sum
device_id                                          
93                          257770  0.118191  30466
296                         247883  0.197993  49079

Load base XGB

In [4]:
base_clf = XGBClassifier()  # init model
base_clf.load_model('app/models/base_model.json')

In [5]:
params = {'max_depth':3, 
#                     'learning_rate':0.3, 
                    'objective':'binary:logistic', 
                    'gamma':3, 
                    'min_child_weight':0.3,
                    'max_delta_step':1,
                    'subsample':1, 
                    'colsample_bytree':0.6, 
                    'colsample_bylevel':0.6, 
                    'random_state':0, 
                    'seed':10, 
                    'n_estimators':15,
                    'missing':None,
                    'eval_metric':'logloss'}

In [20]:
DEVICE_ID = 93
tmp_df = dfeco.query('device_id == @DEVICE_ID').sort_values('ts')

X = tmp_df.drop(non_model_drops, axis=1)
y = tmp_df['ind_shutdown_mal_3D_next']

dfeco_train, dfeco_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [34]:
params = {'max_depth':3, 
#                     'learning_rate':0.3, 
                    'objective':'binary:logistic', 
                    'gamma':0, 
                    'min_child_weight':0.3,
                    'max_delta_step':1,
                    'subsample':1, 
                    'colsample_bytree':0.6, 
                    'colsample_bylevel':0.6, 
#                     'reg_alpha':0, 
#                     'reg_lambda':1, 
 
                    'random_state':0, 
                    'seed':10, 
                    'n_estimators':10,
                    'missing':None,
                    'eval_metric':'logloss'}

clf_93 = XGBClassifier(**params)
# clf3.n_estimators =10
clf_93.fit(dfeco_train, y_train, 
        eval_set=[(dfeco_train, y_train), (dfeco_test, y_test)], 
        xgb_model='app/models/base_model.json', 
        verbose=True,
       )


clf_nobase_93 = XGBClassifier(**params)
# clf3.n_estimators =10
clf_nobase_93.fit(dfeco_train, y_train, 
        eval_set=[(dfeco_train, y_train), (dfeco_test, y_test)], 
#         xgb_model='app/models/base_model.json', 
        verbose=True,
       )

[0]	validation_0-logloss:0.51368	validation_1-logloss:0.51410
[1]	validation_0-logloss:0.46921	validation_1-logloss:0.46968
[2]	validation_0-logloss:0.42114	validation_1-logloss:0.42142
[3]	validation_0-logloss:0.38986	validation_1-logloss:0.39052
[4]	validation_0-logloss:0.36486	validation_1-logloss:0.36555
[5]	validation_0-logloss:0.34373	validation_1-logloss:0.34406
[6]	validation_0-logloss:0.32616	validation_1-logloss:0.32678
[7]	validation_0-logloss:0.30895	validation_1-logloss:0.30932
[8]	validation_0-logloss:0.29551	validation_1-logloss:0.29582
[9]	validation_0-logloss:0.28054	validation_1-logloss:0.28047
[0]	validation_0-logloss:0.59830	validation_1-logloss:0.59780
[1]	validation_0-logloss:0.51261	validation_1-logloss:0.51218
[2]	validation_0-logloss:0.44648	validation_1-logloss:0.44598
[3]	validation_0-logloss:0.38937	validation_1-logloss:0.38866
[4]	validation_0-logloss:0.34381	validation_1-logloss:0.34242
[5]	validation_0-logloss:0.31074	validation_1-logloss:0.30917
[6]	vali

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.6,
              colsample_bynode=1, colsample_bytree=0.6, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=1, max_depth=3, min_child_weight=0.3, missing=None,
              monotone_constraints='()', n_estimators=10, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, seed=10, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [35]:
print(DEVICE_ID)
display(asses_raking(base_clf, y_test, dfeco_test, dfeco.query('device_id == @DEVICE_ID')))
display(asses_raking(clf_nobase_93, y_test, dfeco_test, dfeco.query('device_id == @DEVICE_ID')))
display(asses_raking(clf_93, y_test, dfeco_test, dfeco.query('device_id == @DEVICE_ID')))

296


rank_pct      prob ind_shutdown_mal_3D_next                               
                mean                    count sum      mean cumsum cumsumpct
0        0  0.004944                        8   0  0.000000      0  0.000000
1        1  0.021381                        7   1  0.142857      1  0.066667
2        2  0.095753                        7   3  0.428571      4  0.266667
3        3  0.119614                        7   1  0.142857      5  0.333333
4        4  0.157402                        7   0  0.000000      5  0.333333
5        5  0.209181                        7   2  0.285714      7  0.466667
6        6  0.242208                        7   4  0.571429     11  0.733333
7        7  0.262731                        7   1  0.142857     12  0.800000
8        8  0.297342                        7   3  0.428571     15  1.000000
9        9  0.430800                        7   0  0.000000     15  1.000000

rank_pct      prob ind_shutdown_mal_3D_next                               
                mean                    count sum      mean cumsum cumsumpct
0        0  0.047769                        8   0  0.000000      0  0.000000
1        1  0.056341                        7   0  0.000000      0  0.000000
2        2  0.065936                        7   0  0.000000      0  0.000000
3        3  0.079618                        7   0  0.000000      0  0.000000
4        4  0.131682                        7   0  0.000000      0  0.000000
5        5  0.145212                        7   0  0.000000      0  0.000000
6        6  0.183913                        7   0  0.000000      0  0.000000
7        7  0.335932                        7   2  0.285714      2  0.133333
8        8  0.591297                        7   7  1.000000      9  0.600000
9        9  0.746590                        7   6  0.857143     15  1.000000

rank_pct      prob ind_shutdown_mal_3D_next                               
                mean                    count sum      mean cumsum cumsumpct
0        0  0.010087                        8   0  0.000000      0  0.000000
1        1  0.037253                        7   0  0.000000      0  0.000000
2        2  0.047980                        7   0  0.000000      0  0.000000
3        3  0.067588                        7   0  0.000000      0  0.000000
4        4  0.084502                        7   0  0.000000      0  0.000000
5        5  0.109261                        7   0  0.000000      0  0.000000
6        6  0.198916                        7   0  0.000000      0  0.000000
7        7  0.300165                        7   5  0.714286      5  0.333333
8        8  0.430967                        7   4  0.571429      9  0.600000
9        9  0.575836                        7   6  0.857143     15  1.000000

In [23]:
DEVICE_ID = 296

tmp_df = dfeco.query('device_id == @DEVICE_ID').sort_values('ts')

X = tmp_df.drop(non_model_drops, axis=1)
y = tmp_df['ind_shutdown_mal_3D_next']

dfeco_train, dfeco_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [24]:
clf_296 = XGBClassifier(**params)
clf_296.fit(dfeco_train, y_train, 
        eval_set=[(dfeco_train, y_train), (dfeco_test, y_test)], 
        xgb_model='app/models/base_model.json', 
        verbose=True,
       )

print(DEVICE_ID)
display(asses_raking(base_clf, y_test, dfeco_test, dfeco.query('device_id == @DEVICE_ID')))
display(asses_raking(clf_296, y_test, dfeco_test, dfeco.query('device_id == @DEVICE_ID')))

[0]	validation_0-logloss:0.51368	validation_1-logloss:0.51410
[1]	validation_0-logloss:0.46921	validation_1-logloss:0.46968
[2]	validation_0-logloss:0.42114	validation_1-logloss:0.42142
[3]	validation_0-logloss:0.38986	validation_1-logloss:0.39052
[4]	validation_0-logloss:0.36486	validation_1-logloss:0.36555
[5]	validation_0-logloss:0.34373	validation_1-logloss:0.34406
[6]	validation_0-logloss:0.32616	validation_1-logloss:0.32678
[7]	validation_0-logloss:0.30895	validation_1-logloss:0.30932
[8]	validation_0-logloss:0.29551	validation_1-logloss:0.29582
[9]	validation_0-logloss:0.28054	validation_1-logloss:0.28047
296


rank_pct      prob ind_shutdown_mal_3D_next                               
                mean                    count sum      mean cumsum cumsumpct
0        0  0.004944                        8   0  0.000000      0  0.000000
1        1  0.021381                        7   1  0.142857      1  0.066667
2        2  0.095753                        7   3  0.428571      4  0.266667
3        3  0.119614                        7   1  0.142857      5  0.333333
4        4  0.157402                        7   0  0.000000      5  0.333333
5        5  0.209181                        7   2  0.285714      7  0.466667
6        6  0.242208                        7   4  0.571429     11  0.733333
7        7  0.262731                        7   1  0.142857     12  0.800000
8        8  0.297342                        7   3  0.428571     15  1.000000
9        9  0.430800                        7   0  0.000000     15  1.000000

rank_pct      prob ind_shutdown_mal_3D_next                               
                mean                    count sum      mean cumsum cumsumpct
0        0  0.010087                        8   0  0.000000      0  0.000000
1        1  0.037253                        7   0  0.000000      0  0.000000
2        2  0.047980                        7   0  0.000000      0  0.000000
3        3  0.067588                        7   0  0.000000      0  0.000000
4        4  0.084502                        7   0  0.000000      0  0.000000
5        5  0.109261                        7   0  0.000000      0  0.000000
6        6  0.198916                        7   0  0.000000      0  0.000000
7        7  0.300165                        7   5  0.714286      5  0.333333
8        8  0.430967                        7   4  0.571429      9  0.600000
9        9  0.575836                        7   6  0.857143     15  1.000000

In [28]:
10/15 -1

-0.33333333333333337

In [29]:
tmp[]

,ind_shutdown_mal_3D_next,prob
458771,0,0.144275
348069,0,0.091021
423623,0,0.032905
455804,0,0.086775
504159,0,0.102984
...,...,...
478648,0,0.133193
151161,1,0.539926
93522,0,0.041008
255157,1,0.306504


In [31]:
def highest_20_pct(clf, df, y):
    tmp = pd.DataFrame(y)
    tmp['prob'] = clf.predict_proba(dfeco_test)[:, 1]
    tmp = tmp.loc[(df.tsHour == 0) & (df.tsMinute == 0), :]
    tmp.sort_values('prob', ascending=False, inplace=True)
    return tmp.head(int(0.2*(len(tmp))))['ind_shutdown_mal_3D_next'].sum() / tmp['ind_shutdown_mal_3D_next'].sum()

In [32]:
high_20(clf_93, dfeco.query('device_id == 93'), y_test)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [33]:
high_20(clf_296, dfeco.query('device_id == 296'), y_test)

0.6666666666666666